### Against Win Rate

Computes win rate against opponent.

In [3]:
import pandas as pd

games_df = pd.read_csv("../../data/processed/v3.csv")

games_df.sort_values(by=['date'], inplace=True, ignore_index=True)
games_df

,win_name,lose_name,win_color,lose_color,date,win_rank,lose_rank,komi,PS,OS,streak,PWR,OWR,PNG,ONG,WWHR,WWHR-Error,LWHR,LWHR-Error
0,Kato Shin,Sekiyama Riichi,b,w,1941-05-18,7p,6p,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,Sekiyama Riichi,Kato Shin,b,w,1941-05-27,6p,7p,0.0,-1.0,1.0,-1.0,0.000000,1.000000,1.0,1.0,0.0,0.0,0.0,0.0
2,Kato Shin,Sekiyama Riichi,b,w,1941-06-09,7p,6p,0.0,-1.0,1.0,1.0,0.500000,0.500000,2.0,2.0,-63.0,51.0,-84.0,45.0
3,Sekiyama Riichi,Kato Shin,b,w,1941-06-21,6p,7p,0.0,-1.0,1.0,-1.0,0.333333,0.666667,3.0,3.0,-84.0,44.0,-63.0,51.0
4,Kato Shin,Sekiyama Riichi,b,w,1941-07-01,7p,6p,0.0,-1.0,1.0,1.0,0.500000,0.500000,4.0,4.0,-63.0,51.0,-85.0,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101404,Kim Sangcheon,Lee Hyunho,w,b,2022-02-19,4p,6p,6.5,-1.0,2.0,-1.0,0.576271,0.451613,59.0,62.0,140.0,18.0,148.0,25.0
101405,Shin Minjun,Lee Changseok,w,b,2022-02-19,9p,8p,6.5,-1.0,-1.0,-1.0,0.568627,0.555066,510.0,227.0,339.0,10.0,368.0,12.0
101406,Song Jihoon,Moon Minjong,w,b,2022-02-19,7p,5p,6.5,-1.0,-4.0,-4.0,0.480663,0.505376,181.0,93.0,155.0,19.0,145.0,14.0
101407,Lee Yeongkyu,Park Sangjin,w,b,2022-02-19,9p,6p,6.5,5.0,1.0,1.0,0.565561,0.511450,633.0,131.0,246.0,16.0,246.0,14.0


In [4]:
games_df = games_df.rename(columns={'win_name': 'player', 'lose_name':'opponent'})

In [5]:
from tqdm import tqdm

"""
    Compute every player's cumulative win rates agaisnt opponent
    
    AWR = Against Win Rate
"""


def computeAWR(x):
    n = x["num_games"]
    for i in range(0, x["num_games"] + 1):
        if (n - i) - i == x["AS"]:
            return (n - i) / n if x["reverted"] == 1 else i / n


games_df["AWR"] = 0
against_df = games_df.copy(deep=True)
against_df["match"] = (
    against_df[["player", "opponent"]].apply(sorted, axis=1).astype(str)
)

against_df = against_df.groupby("match")

for name, group in tqdm(against_df):
    group["tolist"] = [
        str(val) for val in group[["player", "opponent"]].values.tolist()
    ]
    group["reverted"] = group.apply(lambda x: 1 if x["tolist"] == name else -1, axis=1)
    group["num_games"] = range(1, len(group) + 1)
    group["AS"] = group["reverted"].rolling(5, min_periods=1).sum()
    group["AWR"] = group.apply(computeAWR, axis=1)

    group = group["AWR"]
    
    group = group.shift()
    games_df.update(group)

games_df

100%|██████████| 43249/43249 [16:52<00:00, 42.72it/s]


,player,opponent,win_color,lose_color,date,win_rank,lose_rank,komi,PS,OS,streak,PWR,OWR,PNG,ONG,WWHR,WWHR-Error,LWHR,LWHR-Error,AWR
0,Kato Shin,Sekiyama Riichi,b,w,1941-05-18,7p,6p,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,Sekiyama Riichi,Kato Shin,b,w,1941-05-27,6p,7p,0.0,-1.0,1.0,-1.0,0.000000,1.000000,1.0,1.0,0.0,0.0,0.0,0.0,1.000000
2,Kato Shin,Sekiyama Riichi,b,w,1941-06-09,7p,6p,0.0,-1.0,1.0,1.0,0.500000,0.500000,2.0,2.0,-63.0,51.0,-84.0,45.0,0.500000
3,Sekiyama Riichi,Kato Shin,b,w,1941-06-21,6p,7p,0.0,-1.0,1.0,-1.0,0.333333,0.666667,3.0,3.0,-84.0,44.0,-63.0,51.0,0.666667
4,Kato Shin,Sekiyama Riichi,b,w,1941-07-01,7p,6p,0.0,-1.0,1.0,1.0,0.500000,0.500000,4.0,4.0,-63.0,51.0,-85.0,44.0,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101404,Kim Sangcheon,Lee Hyunho,w,b,2022-02-19,4p,6p,6.5,-1.0,2.0,-1.0,0.576271,0.451613,59.0,62.0,140.0,18.0,148.0,25.0,1.000000
101405,Shin Minjun,Lee Changseok,w,b,2022-02-19,9p,8p,6.5,-1.0,-1.0,-1.0,0.568627,0.555066,510.0,227.0,339.0,10.0,368.0,12.0,0.000000
101406,Song Jihoon,Moon Minjong,w,b,2022-02-19,7p,5p,6.5,-1.0,-4.0,-4.0,0.480663,0.505376,181.0,93.0,155.0,19.0,145.0,14.0,0.000000
101407,Lee Yeongkyu,Park Sangjin,w,b,2022-02-19,9p,6p,6.5,5.0,1.0,1.0,0.565561,0.511450,633.0,131.0,246.0,16.0,246.0,14.0,0.000000


In [8]:
games_df.loc[(games_df['player'] == 'Shin Jinseo') & (games_df['opponent'] == 'Park Junghwan')]

,player,opponent,win_color,lose_color,date,win_rank,lose_rank,komi,PS,OS,streak,PWR,OWR,PNG,ONG,WWHR,WWHR-Error,LWHR,LWHR-Error,AWR
69555,Shin Jinseo,Park Junghwan,w,b,2016-05-19,5p,9p,6.5,3.0,2.0,3.0,0.584906,0.700647,159.0,618.0,425.0,6.0,484.0,5.0,1.000000
75600,Shin Jinseo,Park Junghwan,b,w,2017-07-21,8p,9p,6.5,1.0,9.0,1.0,0.641350,0.702550,237.0,706.0,492.0,6.0,524.0,6.0,0.800000
76883,Shin Jinseo,Park Junghwan,b,w,2017-10-05,8p,9p,6.5,-2.0,-2.0,-2.0,0.647059,0.703039,255.0,724.0,495.0,6.0,530.0,5.0,0.000000
78624,Shin Jinseo,Park Junghwan,b,w,2018-02-10,8p,9p,6.5,1.0,14.0,1.0,0.653430,0.710145,277.0,759.0,505.0,6.0,533.0,5.0,0.000000
90038,Shin Jinseo,Park Junghwan,w,b,2020-02-10,9p,9p,6.5,17.0,-1.0,17.0,0.702586,0.713981,464.0,937.0,668.0,6.0,534.0,6.0,0.000000
90052,Shin Jinseo,Park Junghwan,b,w,2020-02-12,9p,9p,6.5,18.0,-2.0,18.0,0.703226,0.713220,465.0,938.0,668.0,6.0,534.0,6.0,0.421053
91183,Shin Jinseo,Park Junghwan,b,w,2020-05-03,9p,9p,6.5,3.0,2.0,3.0,0.708940,0.712474,481.0,946.0,683.0,6.0,535.0,6.0,0.000000
91558,Shin Jinseo,Park Junghwan,b,w,2020-06-15,9p,9p,6.5,3.0,2.0,3.0,0.711968,0.712487,493.0,953.0,689.0,6.0,538.0,6.0,0.000000
91626,Shin Jinseo,Park Junghwan,w,b,2020-06-22,9p,9p,6.5,4.0,-1.0,4.0,0.712551,0.711740,494.0,954.0,690.0,6.0,537.0,6.0,0.565217
91629,Shin Jinseo,Park Junghwan,b,w,2020-06-23,9p,9p,6.5,5.0,-2.0,5.0,0.713131,0.710995,495.0,955.0,691.0,6.0,537.0,6.0,0.000000
